# **Building and Training a Feedforward Neural Network for Language Modeling**


### Importing The Required Libraries

In [1]:
%%capture
import warnings
from tqdm import tqdm

warnings.simplefilter('ignore')
import time
from collections import OrderedDict

import re

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import string
import time
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from nltk.tokenize import word_tokenize




### Defining Preprocessing Functions 

In [2]:
def preprocess_string(s):
    """
    Preprocesses a given string by performing the following steps:
    
    1. Removes all non-word characters (excluding letters and numbers).
    2. Removes all whitespace characters.
    3. Removes all numeric digits.

    Parameters:
    s (str): The input string to be cleaned.

    Returns:
    str: The processed string with only alphabetic characters, no spaces, and no digits.
    """

    # Remove all non-word characters (everything except letters and numbers)
    # \w matches any word character (letters, numbers, and underscores)
    # \s matches any whitespace characters
    # ^ inside [] negates the selection, so [^\w\s] matches anything that's NOT a word character or whitespace.
    s = re.sub(r"[^\w\s]", '', s)

    # Remove all whitespace characters (spaces, tabs, newlines)
    # \s+ matches one or more whitespace characters.
    s = re.sub(r"\s+", '', s)

    # Remove all digits (0-9)
    # \d matches any digit character.
    s = re.sub(r"\d", '', s)

    return s

In [3]:
def process(word):
    tokens=word_tokenize(word)
    tokens=[preprocess_string(token) for token in tokens]
    tokens=[token.lower() for token in tokens]
    return tokens

In [4]:
def convert_to_indices(tokens):
    vocab = build_vocab_from_iterator([tokens],specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])
    return vocab



### Giving Input 

In [5]:
song= """We are no strangers to love
You know the rules and so do I
A full commitments what Im thinking of
You wouldnt get this from any other guy
I just wanna tell you how Im feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Weve known each other for so long
Your hearts been aching but youre too shy to say it
Inside we both know whats been going on
We know the game and were gonna play it
And if you ask me how Im feeling
Dont tell me youre too blind to see
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Weve known each other for so long
Your hearts been aching but youre too shy to say it
Inside we both know whats been going on
We know the game and were gonna play it
I just wanna tell you how Im feeling
Gotta make you understand
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up
Never gonna let you down
Never gonna run around and desert you
Never gonna make you cry
Never gonna say goodbye
Never gonna tell a lie and hurt you"""

### Preprocessing The Input song

In [6]:
tokens=process(song)

In [7]:
vocab=convert_to_indices(tokens)

In [8]:
for token in tokens[0:10]:
    print(vocab[token]) 

22
60
72
76
26
71
3
21
32
74


In [9]:
def get_embedding(vocab):
    embedding_dim=20
    vocab_size=len(vocab)
    return nn.Embedding(vocab_size,embedding_dim)

In [10]:
embeddings=get_embedding(vocab)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\rajve\miniconda3\envs\nlp_env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    a

In [11]:
index_to_token=vocab.get_itos()
for n in range(2): 
    embedding=embeddings(torch.tensor(n))
    print("word",index_to_token[n])
    print("index",n)
    print( "embedding", embedding)
    print("embedding shape", embedding.shape)

word <unk>
index 0
embedding tensor([-0.9882,  1.9309,  0.1308, -0.3276,  1.3347,  0.8058,  0.6635,  0.5976,
         1.5102,  0.8893, -1.0650,  0.0569,  1.1009,  1.0863,  1.1724,  0.1959,
        -0.6445,  1.6000, -0.0959,  2.1565], grad_fn=<EmbeddingBackward0>)
embedding shape torch.Size([20])
word na
index 1
embedding tensor([-0.4959,  0.8139,  0.1411, -1.0038,  0.2841, -1.8769, -1.4475, -0.1214,
        -0.4777, -1.2710,  0.0426,  1.4992,  0.5301,  1.3597, -0.4525,  0.6394,
        -0.3507,  1.7663,  0.5696,  1.6529], grad_fn=<EmbeddingBackward0>)
embedding shape torch.Size([20])


### Defining The Model

In [12]:
class NGramLanguageModel(nn.Module):
    def __init__(self,vocab_size,embed_dim):
        self.embed=nn.Embedding(vocab_size,embed_dim)
        self.linear1=torch.relu(nn.Linear(Context_Size*embed_dim,128))
        self.linear2=torch.relu(nn.Linear(128,64))
        self.out=torch.softmax(nn.Linear(64,vocab_size))
    def call(self,inp_indx):
        inp_embed=self.embed(inp_indx)
        outp=self.linear1(inp_embed)
        outp=self.linear2(outp)
        outp=self.out(outp)
        return outp
        

In [13]:
def give_final_indx(out):
    return torch.argmax(out)

In [14]:
def give_final_string(idx):
    return index_to_token(idx)

### Structuring The Input before passing it to the Model

In [19]:
Context_Size=2
input_structured=[
                    (
                      [vocab[tokens[i-j-1]] for j in range(Context_Size)],
                       vocab[tokens[i]])
                      for i in range(Context_Size,len(vocab))
                    ]

In [20]:
input_structured[0]

([60, 22], 72)

In [22]:
from torch.utils.data import DataLoader
dataloader=DataLoader(input_structured,batch_size=2,shuffle=False)

In [24]:
for context,target in dataloader:
    print("Context:",context)
    print("Target:",target)

Context: [tensor([60, 72]), tensor([22, 60])]
Target: tensor([72, 76])
Context: [tensor([76, 26]), tensor([72, 76])]
Target: tensor([26, 71])
Context: [tensor([71,  3]), tensor([26, 71])]
Target: tensor([ 3, 21])
Context: [tensor([21, 32]), tensor([ 3, 21])]
Target: tensor([32, 74])
Context: [tensor([74,  5]), tensor([32, 74])]
Target: tensor([ 5, 31])
Context: [tensor([31, 64]), tensor([ 5, 31])]
Target: tensor([64, 29])
Context: [tensor([29,  9]), tensor([64, 29])]
Target: tensor([ 9, 67])
Context: [tensor([67, 63]), tensor([ 9, 67])]
Target: tensor([63, 79])
Context: [tensor([79, 24]), tensor([63, 79])]
Target: tensor([24, 77])
Context: [tensor([77, 73]), tensor([24, 77])]
Target: tensor([73,  3])
Context: [tensor([ 3, 80]), tensor([73,  3])]
Target: tensor([80, 68])
Context: [tensor([68, 78]), tensor([80, 68])]
Target: tensor([78, 66])
Context: [tensor([66, 59]), tensor([78, 66])]
Target: tensor([59, 30])
Context: [tensor([30, 69]), tensor([59, 30])]
Target: tensor([69, 29])
Contex